In [1]:
!pip install fastapi uvicorn pandas openai sentence-transformers torch transformers kagglehub pyngrok python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("syedmharis/software-engineering-interview-questions-dataset")
name = 'Software Questions.csv'
dataset_path = f'{path}/{name}'

print("Path to dataset files:", dataset_path)

100%|██████████| 12.0k/12.0k [00:00<00:00, 16.6MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/syedmharis/software-engineering-interview-questions-dataset/versions/1/Software Questions.csv


In [3]:
from fastapi import FastAPI, HTTPException, Query
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import pandas as pd
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util
import random

# Initialize the FastAPI app
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load the dataset
df = pd.read_csv(dataset_path, encoding='ISO-8859-1')

# Configure OpenAI API
XAI_API_KEY = ""
client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)

# Load pre-trained Sentence Transformer model for similarity
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')

# Pydantic Models
class FeedbackRequest(BaseModel):
    user_answer: str
    question_number: int
    question: str

# Endpoint to get a random question by category and difficulty
@app.get('/get_question')
def get_question(category: str = Query(None), difficulty: str = Query(None)):
    filtered_df = df
    if category:
        filtered_df = filtered_df[filtered_df['Category'].str.contains(category, case=False, na=False)]
    if difficulty:
        filtered_df = filtered_df[filtered_df['Difficulty'].str.contains(difficulty, case=False, na=False)]

    if not filtered_df.empty:
        question = filtered_df.sample(1).iloc[0]
        return {
            'question_number': int(question['Question Number']),
            'question': question['Question'],
            'category': question['Category'],
            'difficulty': question['Difficulty']
        }
    else:
        raise HTTPException(status_code=404, detail='No questions found for the specified filters.')

# Endpoint to get AI feedback for a user's answer
@app.post('/get_feedback')
def get_feedback(request: FeedbackRequest):
    question_number = request.question_number
    user_answer = request.user_answer
    question = request.question

    # Retrieve the correct answer
    correct_answer = df[df['Question Number'] == question_number]['Answer'].values
    if len(correct_answer) == 0:
        raise HTTPException(status_code=404, detail='Question not found.')

    correct_answer = correct_answer[0]

    # Generate feedback using OpenAI API
    try:
        prompt = (f"Question: {question}\n"
                  f"Correct Answer: {correct_answer}\n"
                  f"User's Answer: {user_answer}\n"
                  "Provide constructive feedback for the user's answer, focusing on improvements and strengths.")

        completion = client.chat.completions.create(
            model="grok-2-1212",
            messages=[
                {"role": "system", "content": "You are an expert technical interviewer providing detailed feedback."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.5
        )

        feedback = completion.choices[0].message.content

        # Calculate similarity using Sentence Transformer
        embeddings = similarity_model.encode([user_answer, correct_answer])
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()

        return {'feedback': feedback, 'correct': similarity > 60, 'similarity': similarity}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Endpoint to list categories
@app.get('/list_categories')
def list_categories():
    categories = df['Category'].dropna().unique().tolist()
    return {'categories': categories}

# Endpoint to list difficulties
@app.get('/list_difficulties')
def list_difficulties():
    difficulties = df['Difficulty'].dropna().unique().tolist()
    return {'difficulties': difficulties}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
!ngrok config add-authtoken 

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
# Save the FastAPI server code into a file named server.py

code = '''
from fastapi import FastAPI, HTTPException, Query
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import pandas as pd
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util
import random

# Initialize the FastAPI app
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load the dataset
df = pd.read_csv("/root/.cache/kagglehub/datasets/syedmharis/software-engineering-interview-questions-dataset/versions/1/Software Questions.csv", encoding="ISO-8859-1")

# Configure OpenAI API
XAI_API_KEY = ""
client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)

# Load pre-trained Sentence Transformer model for similarity
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')

# Pydantic Models
class FeedbackRequest(BaseModel):
    user_answer: str
    question_number: int
    question: str

# Endpoint to get a random question by category and difficulty
@app.get('/get_question')
def get_question(category: str = Query(None), difficulty: str = Query(None)):
    filtered_df = df
    if category:
        filtered_df = filtered_df[filtered_df['Category'].str.contains(category, case=False, na=False)]
    if difficulty:
        filtered_df = filtered_df[filtered_df['Difficulty'].str.contains(difficulty, case=False, na=False)]

    if not filtered_df.empty:
        question = filtered_df.sample(1).iloc[0]
        return {
            'question_number': int(question['Question Number']),
            'question': question['Question'],
            'category': question['Category'],
            'difficulty': question['Difficulty']
        }
    else:
        raise HTTPException(status_code=404, detail='No questions found for the specified filters.')

# Endpoint to get AI feedback for a user's answer
@app.post('/get_feedback')
def get_feedback(request: FeedbackRequest):
    question_number = request.question_number
    user_answer = request.user_answer
    question = request.question

    # Retrieve the correct answer
    correct_answer = df[df['Question Number'] == question_number]['Answer'].values
    if len(correct_answer) == 0:
        raise HTTPException(status_code=404, detail='Question not found.')

    correct_answer = correct_answer[0]

    # Generate feedback using OpenAI API
    try:
        prompt = (f"Question: {question}\\n"
                  f"Correct Answer: {correct_answer}\\n"
                  f"User's Answer: {user_answer}\\n"
                  "Provide constructive feedback for the user's answer, focusing on improvements and strengths.")

        completion = client.chat.completions.create(
            model="grok-2-1212",
            messages=[
                {"role": "system", "content": "You are an expert technical interviewer providing detailed feedback."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.5
        )

        feedback = completion.choices[0].message.content

        # Calculate similarity using Sentence Transformer
        embeddings = similarity_model.encode([user_answer, correct_answer])
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()

        return {'feedback': feedback, 'correct': similarity > 0.60, 'similarity': similarity}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Endpoint to list categories
@app.get('/list_categories')
def list_categories():
    categories = df['Category'].dropna().unique().tolist()
    return {'categories': categories}

# Endpoint to list difficulties
@app.get('/list_difficulties')
def list_difficulties():
    difficulties = df['Difficulty'].dropna().unique().tolist()
    return {'difficulties': difficulties}
'''

with open("server.py", "w") as f:
    f.write(code)

In [6]:
from pyngrok import ngrok
import subprocess
import time

# Run FastAPI server in the background
server_process = subprocess.Popen(["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"])

# Give the server a few seconds to start
time.sleep(3)

# Create a public URL using ngrok
public_url = ngrok.connect(8000).public_url
print(f"🚀 FastAPI is running at: {public_url}")

🚀 FastAPI is running at: https://0d41-34-19-78-133.ngrok-free.app


In [ ]:
# Terminate the server process and disconnect ngrok
server_process.terminate()
ngrok.disconnect(public_url)